# Three bird classifier
This code is used to train and score the three bird classifier.

# Importing Libraries 📚

In [2]:
import os
import gc
import csv
import ast
import random
import math
import shutil
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
from tqdm import tqdm
import torchaudio
import IPython.display as ipd
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models

import timm

import warnings
warnings.filterwarnings('ignore')

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
class config:
    seed = 2022
    num_fold = 5
    sample_rate = 32_000
    n_fft = 1024
    hop_length = 512
    n_mels = 64
    duration = 5
    num_classes = 4
    train_batch_size = 32
    valid_batch_size = 32
    model_name = 'resnet50'
    # model_name = 'efficientnet_b5'
    epochs = 30
    device = 'mps' if torch.backends.mps.is_available() else 'cpu'
    learning_rate = 1e-4

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(config.seed)

In [5]:
def getRandomFile(path, not_in_dir):
    randomDir = random.choice([(x) for x in list(os.scandir(path)) if x.is_dir()]).name
    if randomDir in not_in_dir:
        return getRandomFile(path, not_in_dir)
    randomFile = random.choice([f for f in list(os.scandir(os.path.join(path, randomDir)))]).name
    return os.path.join(path, randomDir, randomFile)

<a id='2'></a>
# Reading the data 📖

In [6]:
root_dir = 'data/audio/'
class_names = ['galerida_cristata', 'picus_viridis', 'phasianus_colchicus', 'random_other']
# class_names = ['galerida_cristata', 'picus_viridis', 'phasianus_colchicus']
random_files = []

In [40]:
for i in range(352):
    random_files.append(getRandomFile(root_dir, class_names))

In [41]:
random_files = list(set(random_files))

In [42]:
for file in random_files:
    shutil.copy(file, 'data/audio/random_other')

In [6]:
files_df = pd.DataFrame(columns=['file_path', 'observation', 'target'])

In [64]:
for class_name in class_names:
    files_path = os.path.join(root_dir, class_name, 'ogg_5s')
    file_paths = []
    observations = []
    
    for file in os.listdir(files_path):
        observation = file.split(' ')[0]
        number = file.split('_')[1].split('.')[0]
        
        # Include the first minute for each file
        if int(number) <= 3:
            file_paths.append(os.path.join(files_path, file))
            observations.append(observation)
    
    #file_paths = [os.path.join(files_path, file) for file in os.listdir(files_path)]
    
    df = pd.DataFrame(list(zip(file_paths, observations)), columns=['file_path', 'observation'])
    df['target'] = class_name
    
    files_df = pd.concat([files_df, df], ignore_index=True)

In [7]:
files_df = pd.DataFrame(columns=['file_path', 'target'])

In [8]:
for class_name in class_names:
    files_path = os.path.join(root_dir, class_name, 'ogg')
    file_paths = [os.path.join(files_path, file) for file in os.listdir(files_path)]
    
    df = pd.DataFrame(file_paths, columns=['file_path'])
    df['target'] = class_name
    
    files_df = pd.concat([files_df, df], ignore_index=True)

In [1]:
# files_df[files_df['observation'] == 'XC630463']

In [9]:
files_df.groupby(['target']).count()

,file_path
target,
galerida_cristata,311
phasianus_colchicus,258
picus_viridis,487
random_other,345


<a id='4'></a>
# Dataset Preprocessing 🛠️

### Convert the target variable from string to integer using the LabelEncoder

In [10]:
encoder = LabelEncoder()
files_df['target_encoded'] = encoder.fit_transform(files_df['target'])

In [11]:
files_df.groupby(['target', 'target_encoded']).count()

,,file_path
target,target_encoded,
galerida_cristata,0,311
phasianus_colchicus,1,258
picus_viridis,2,487
random_other,3,345


### Next create folds.
When using multiple audio segments from one file, we want to use the group fold to keep them together and prevent overfitting. Otherwise, use the Stratified K Fold

In [69]:
# skf = StratifiedKFold(n_splits=config.num_fold)
skf = GroupKFold(n_splits=config.num_fold)

for k, (_, val_ind) in enumerate(skf.split(X=files_df, y=files_df['target_encoded'], groups=files_df['observation'])):
    files_df.loc[val_ind, 'fold'] = k

In [12]:
skf = StratifiedKFold(n_splits=config.num_fold)

for k, (_, val_ind) in enumerate(skf.split(X=files_df, y=files_df['target_encoded'])):
    files_df.loc[val_ind, 'fold'] = k

In [13]:
files_df.groupby(['fold', 'target', 'target_encoded']).count()

file_path
fold target              target_encoded           
0.0  galerida_cristata   0                      63
     phasianus_colchicus 1                      52
     picus_viridis       2                      97
     random_other        3                      69
1.0  galerida_cristata   0                      62
     phasianus_colchicus 1                      51
     picus_viridis       2                      98
     random_other        3                      69
2.0  galerida_cristata   0                      62
     phasianus_colchicus 1                      51
     picus_viridis       2                      98
     random_other        3                      69
3.0  galerida_cristata   0                      62
     phasianus_colchicus 1                      52
     picus_viridis       2                      97
     random_other        3                      69
4.0  galerida_cristata   0                      62
     phasianus_colchicus 1                      52
     picus_viridis       2                      97
     random_other        3                      69

In [14]:
files_df

,file_path,target,target_encoded,fold
0,data/audio/galerida_cristata/ogg/XC155090 - Cr...,galerida_cristata,0,0.0
1,data/audio/galerida_cristata/ogg/XC293927 - Cr...,galerida_cristata,0,0.0
2,data/audio/galerida_cristata/ogg/XC522185 - Cr...,galerida_cristata,0,0.0
3,data/audio/galerida_cristata/ogg/XC387237 - Cr...,galerida_cristata,0,0.0
4,data/audio/galerida_cristata/ogg/XC46297 - Cre...,galerida_cristata,0,0.0
...,...,...,...,...
1396,data/audio/random_other/ogg/XC568005 - Eurasia...,random_other,3,4.0
1397,data/audio/random_other/ogg/XC495312 - Eurasia...,random_other,3,4.0
1398,data/audio/random_other/ogg/XC133194 - Red-bac...,random_other,3,4.0
1399,data/audio/random_other/ogg/XC162216 - Long-ta...,random_other,3,4.0


In [14]:
mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate=config.sample_rate, 
                                                  n_fft=config.n_fft, 
                                                  hop_length=config.hop_length, 
                                                  n_mels=config.n_mels)

### So similarly we will extract mel spectrogram for each audio and will train the model using them. But wait, this is not the end. There are several things which we need to consider before extracting spectrograms from the audio files. We want our dataset to be uniform and to do that we should consider the below points:-
* As I mentioned above the audio data consist of two things - sample rate and sound. Not all the audio have same sample rate, and this is a huge problem if we want uniformity in the melspectrogram which we extract. So we resample the data so that all the data have same sample rates.
* Next if we talk about the sound, the dimension of sound is - (num_channels, num_samples). If we talk about number of channels, then each audio signals can have different number of channels. So we will ensure that they are mono, i.e., num_channels = 1.
* Lastly, each audio signal have different time durations which lead to difference in number of samples. So we ensure same number of samples by applying padding if it is less than the desired samples or by truncating if it is more than the desired samples.

### Now I will implement the custom Dataset class in which I will also implement all the above points. 

In [30]:
class BirdClefDataset(Dataset):
    def __init__(self, df, transformation, target_sample_rate, duration):
        self.audio_paths = df['file_path'].values
        self.labels = df['target_encoded'].values
        self.transformation = transformation
        self.target_sample_rate = target_sample_rate
        self.num_samples = target_sample_rate*duration
        
    def __len__(self):
        return len(self.audio_paths)
    
    def __getitem__(self, index):
        audio_path = self.audio_paths[index]
        signal, sr = torchaudio.load(audio_path) # loaded the audio
        
        # Now we first checked if the sample rate is same as TARGET_SAMPLE_RATE and if it not equal we perform resampling
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        
        # Next we check the number of channels of the signal
        #signal -> (num_channels, num_samples) - Eg.-(2, 14000) -> (1, 14000)
        if signal.shape[0]>1:
            signal = torch.mean(signal, axis=0, keepdim=True)
        
        # Lastly we check the number of samples of the signal
        #signal -> (num_channels, num_samples) - Eg.-(1, 14000) -> (1, self.num_samples)
        # If it is more than the required number of samples, we truncate the signal
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        
        # If it is less than the required number of samples, we pad the signal
        if signal.shape[1]<self.num_samples:
            num_missing_samples = self.num_samples - signal.shape[1]
            last_dim_padding = (0, num_missing_samples)
            signal = F.pad(signal, last_dim_padding)
        
        # Finally all the process has been done and now we will extract mel spectrogram from the signal
        mel = self.transformation(signal)
        
        # For pretrained models, we need 3 channel image, so for that we concatenate the extracted mel
        image = torch.cat([mel, mel, mel])
        
        # image = mel
        
        # Normalized the image
        max_val = torch.abs(image).max()
        image = image / max_val
        
        label = torch.tensor(self.labels[index])
        
        return image, label

In [16]:
# Function to get data according to the folds
def get_data(fold):    
    train_df = files_df[files_df['fold'] != fold].reset_index(drop=True)
    valid_df = files_df[files_df['fold'] == fold].reset_index(drop=True)
    
    train_dataset = BirdClefDataset(train_df, mel_spectrogram, config.sample_rate, config.duration)
    valid_dataset = BirdClefDataset(valid_df, mel_spectrogram, config.sample_rate, config.duration)

    # dataset = BirdClefDataset(files_df, mel_spectrogram, config.sample_rate, config.duration)
    # train_dataset, valid_dataset = random_split(dataset, [0.7, 0.3])
    
    train_loader = DataLoader(train_dataset, batch_size=config.train_batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=config.valid_batch_size, shuffle=False)
    
    return train_loader, valid_loader

<a id='5'></a>
# Model 🤖

### We fine tune a pretrained model. We use Resnet50 and EfficientNet

In [31]:
class BirdCLEFResnet(nn.Module):
    def __init__(self):
        super(BirdCLEFResnet, self).__init__()
        self.base_model = models.__getattribute__(config.model_name)(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
            
        in_features = self.base_model.fc.in_features
        
        self.base_model.fc = nn.Sequential(
            nn.Linear(in_features, 1024), 
            nn.ReLU(), 
            nn.Dropout(p=0.2),
            nn.Linear(1024, 512), 
            nn.ReLU(), 
            nn.Dropout(p=0.2),
            nn.Linear(512, config.num_classes))
        
    def forward(self, x):
        x = self.base_model(x)
        return x

In [19]:
class BirdCLEFEfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.cfg = config
        self.model = timm.create_model(self.cfg.model_name, pretrained=True, in_chans=1)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(self.n_features, self.cfg.num_classes)

    def forward(self, x):
        output = self.model(x)
        return output

<a id='6'></a>
# Utility Functions 📋

### Next we define some functions to train the model. These are the basic functions which we use to train any pytorch based models.

In [20]:
def loss_fn(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)

def train(model, data_loader, optimizer, scheduler, device, epoch):
    model.train()
    
    running_loss = 0
    loop = tqdm(data_loader, position=0)
    for i, (mels, labels) in enumerate(loop):
        mels = mels.to(device)
        labels = labels.to(device)
        
        outputs = model(mels)
        _, preds = torch.max(outputs, 1)
        
        loss = loss_fn(outputs, labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if scheduler is not None:
            scheduler.step()
            
        running_loss += loss.item()
        
        loop.set_description(f"Epoch [{epoch+1}/{config.epochs}]")
        loop.set_postfix(loss=loss.item())

    return running_loss/len(data_loader)

In [21]:
def valid(model, data_loader, device, epoch):
    model.eval()
    
    running_loss = 0
    pred = []
    label = []
    
    loop = tqdm(data_loader, position=0)
    for mels, labels in loop:
        mels = mels.to(device)
        labels = labels.to(device)
        
        outputs = model(mels)
        _, preds = torch.max(outputs, 1)
        
        loss = loss_fn(outputs, labels)
            
        running_loss += loss.item()
        
        pred.extend(preds.view(-1).cpu().detach().numpy())
        label.extend(labels.view(-1).cpu().detach().numpy())
        
        loop.set_description(f"Epoch [{epoch+1}/{config.epochs}]")
        loop.set_postfix(loss=loss.item())
        
    valid_f1 = f1_score(label, pred, average='macro')
    
    return running_loss/len(data_loader), valid_f1

In [34]:
best_valid_log = []

def run(fold):
    train_loader, valid_loader = get_data(fold)
    
    # model = BirdClefModel().to(config.device) # check version 3 for this
    model = BirdCLEFResnet().to(config.device)
    # model = BirdCLEFEfficientNet().to(config.device)
    
    optimizer = Adam(model.parameters(), lr=config.learning_rate)
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, eta_min=1e-5, T_max=10)
    
    best_valid_f1 = 0
    for epoch in range(config.epochs):
        train_loss = train(model, train_loader, optimizer, scheduler, config.device, epoch)
        valid_loss, valid_f1 = valid(model, valid_loader, config.device, epoch)
        if valid_f1 > best_valid_f1:
            print(f"Validation F1 Improved - {best_valid_f1} ---> {valid_f1}")
            best_valid_log.append({fold, valid_f1})
            
            torch.save(model.state_dict(), f'./model_{fold}.bin')
            print(f"Saved model checkpoint at ./model_{fold}.bin")
            best_valid_f1 = valid_f1
            
    return best_valid_f1

<a id='7'></a>
# Training ⚙️

In [37]:
for fold in range(config.num_fold):
    print("=" * 30)
    print("Training Fold - ", fold)
    print("=" * 30)
    best_valid_f1 = run(fold)
    print(f'Best F1 Score: {best_valid_f1:.5f}')
    
    gc.collect()
    torch.cuda.empty_cache()

Training Fold -  0


Epoch [1/30]: 100%|██████████████████████████████████████████████████████████████████| 9/9 [00:16<00:00,  1.81s/it, loss=1.32]


Validation F1 Improved - 0 ---> 0.1276595744680851
Saved model checkpoint at ./model_0.bin


Epoch [2/30]: 100%|██████████████████████████████████████████████████████████████████| 9/9 [00:18<00:00,  2.00s/it, loss=1.41]


Validation F1 Improved - 0.1276595744680851 ---> 0.22362285544103727
Saved model checkpoint at ./model_0.bin


...

Validation F1 Improved - 0.6443212447994954 ---> 0.6600004025428786
Saved model checkpoint at ./model_4.bin


Epoch [26/30]: 100%|████████████████████████████████████████████████████████████████| 9/9 [00:13<00:00,  1.54s/it, loss=0.928]


Validation F1 Improved - 0.6600004025428786 ---> 0.669474244509979
Saved model checkpoint at ./model_4.bin


Epoch [30/30]: 100%|████████████████████████████████████████████████████████████████| 9/9 [00:14<00:00,  1.57s/it, loss=0.824]


Best F1 Score: 0.66947


In [38]:
best_valid_log

[{0, 0.1276595744680851},
 {0, 0.22362285544103727},
 {0, 0.38844537944671664},
 {0, 0.4940145228800144},
 {0, 0.5825422377133401},
 {0, 0.624826602880038},
 {0, 0.6278509542566986},
 {0, 0.6312790894474063},
 {0.1459441489361702, 1},
 {0.42095571194793646, 1},
 {0.4738256932654216, 1},
 {0.5437813128007429, 1},
 {0.5638704502284512, 1},
 {0.5734064040193079, 1},
 {0.6036310549668228, 1},
 {0.6245176612606939, 1},
 {0.6595166271636861, 1},
 {0.1459441489361702, 2},
 {0.3410296919062199, 2},
 {0.404414535936275, 2},
 {0.5269997606969589, 2},
 {0.5654898909947198, 2},
 {0.5671921456781404, 2},
 {0.5718927022746418, 2},
 {0.5877597540288381, 2},
 {0.595815100362314, 2},
 {0.600434332289963, 2},
 {0.14388985759953501, 3},
 {0.3859951260336297, 3},
 {0.5007224418093983, 3},
 {0.583018399018416, 3},
 {0.6402080376559345, 3},
 {0.6434792232141445, 3},
 {0.6517802033861086, 3},
 {0.6646264124440817, 3},
 {0.6807299197513014, 3},
 {0.6874845092715349, 3},
 {0.1286472148541114, 4},
 {0.306093585

# Score

In [222]:
root_dir = 'data/test/ogg'
score_df = pd.DataFrame(columns=['file_path', 'target'])
file_paths = [os.path.join(root_dir, file) for file in os.listdir(root_dir)]
score_df['file_path'] = file_paths

score_df

,file_path,target
0,data/test/ogg/XC714560 - Kuifleeuwerik - Galer...,NaN
1,data/test/ogg/XC790509 - Groene Specht - Picus...,NaN
2,data/test/ogg/XC784467 - Groene Specht - Picus...,NaN
3,data/test/ogg/XC577099 - Fazant - Phasianus co...,NaN


In [223]:
encoder = LabelEncoder()
score_df['target_encoded'] = encoder.fit_transform(score_df['target'])

In [225]:
score_df

,file_path,target,target_encoded
0,data/test/ogg/XC714560 - Kuifleeuwerik - Galer...,NaN,0
1,data/test/ogg/XC790509 - Groene Specht - Picus...,NaN,0
2,data/test/ogg/XC784467 - Groene Specht - Picus...,NaN,0
3,data/test/ogg/XC577099 - Fazant - Phasianus co...,NaN,0


In [226]:
score_dataset = BirdClefDataset(score_df, mel_spectrogram, config.sample_rate, config.duration)
score_loader = DataLoader(score_dataset, batch_size=1, shuffle=False)

In [ ]:
# Classes
# galerida_cristata		0
# phasianus_colchicus	1
# picus_viridis			2

In [26]:
def generate_model_output(filename, sensor, model, device):
    audio_path = os.path.join(f"data/recordings/ogg_files/{sensor}", filename)
    
    transformation = mel_spectrogram
    target_sample_rate = 32000
    num_samples = target_sample_rate * 5
    
    signals, sr = torchaudio.load(audio_path) # loaded the audio

    # Now we first checked if the sample rate is same as TARGET_SAMPLE_RATE and if it not equal we perform resampling
    if sr != target_sample_rate:
        resampler = torchaudio.transforms.Resample(sr, target_sample_rate)
        signals = resampler(signals)

    # Next we check the number of channels of the signal
    #signal -> (num_channels, num_samples) - Eg.-(2, 14000) -> (1, 14000)
    if signals.shape[0] > 1:
        signals = torch.mean(signals, axis=0, keepdim=True)
        
    chunks = math.ceil(torch.numel(signals) / num_samples)
    
    # opening the csv file in 'w' mode
    file_location = f"data/results_3b/{sensor}/" + filename.split('.')[0] + '.csv'
    file = open(file_location, 'w', newline ='')

    with file:
        # identifying header 
        header = ['frame', 'class', 'probability']
        writer = csv.DictWriter(file, fieldnames = header)

        # writing data row-wise into the csv file
        writer.writeheader()
    
        for i in range(chunks):
            # Split the signal into 5 second segments
            signal = signals[:, i*num_samples:num_samples*(i+1)]

            # Lastly we check the number of samples of the signal
            #signal -> (num_channels, num_samples) - Eg.-(1, 14000) -> (1, self.num_samples)
            # If it is more than the required number of samples, we truncate the signal
            if signal.shape[1] > num_samples:
                signal = signal[:, :num_samples]

            # If it is less than the required number of samples, we pad the signal
            if signal.shape[1] < num_samples:
                num_missing_samples = num_samples - signal.shape[1]
                last_dim_padding = (0, num_missing_samples)
                signal = F.pad(signal, last_dim_padding)

            # Finally all the process has been done and now we will extract mel spectrogram from the signal
            mel = transformation(signal)

            # For pretrained models, we need 3 channel image, so for that we concatenate the extracted mel
            # image = torch.cat([mel, mel, mel])
            
            image = mel

            # Normalized the image
            max_val = torch.abs(image).max()
            image = image / max_val

            # Insert image into model

            # Somehow it needs to go from 3D to 4D
            image = image.unsqueeze(0)

            # To MPS
            image = image.to(device)

            output = model(image)

            prob = F.softmax(output, dim=1)
            top_p, top_class = prob.topk(1, dim = 1)
            
            writer.writerow({'frame' : i,
                             'class': class_names[top_class],
                             'probability': float(top_p)})
            
            if float(top_p) > 0.95:
                print(i, float(top_p), class_names[top_class])

In [25]:
def predict(model, data_loader, device):
    loop = tqdm(data_loader, position=0)
    for mels, labels in loop:
        mels = mels.to(device)
        
        output = model(mels)
        
        prob = F.softmax(output, dim=1)
        top_p, top_class = prob.topk(1, dim = 1)
        
        print(top_p, top_class)
        
        _, preds = torch.max(output, 1)

In [27]:
# model = BirdCLEFResnet().to(config.device)
# model.load_state_dict(torch.load('model_best_more_data.bin'))

model = BirdCLEFEfficientNet().to(config.device)
model.load_state_dict(torch.load('model_4.bin'))
model.eval()

# predict(model, score_loader, config.device)

BirdCLEFEfficientNet(
  (model): EfficientNet(
    (conv_stem): Conv2d(1, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNormAct2d(
            48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, kerne

In [28]:
# Get the list of all files and directories
# sensor = '9_035_217'
sensor = '1_041_209'

path = f"data/recordings/ogg_files/{sensor}"
audio_recordings = os.listdir(path)

audio_recordings_done = [f.replace('csv', 'ogg') for f in os.listdir(f"data/results_3b/{sensor}/")]

audio_recordings_todo = list(set(audio_recordings) - set(audio_recordings_done))
audio_recordings_todo.sort()
audio_recordings_todo

['20221230_143000.ogg',
 '20221230_213000.ogg',
 '20221231_060000.ogg',
 '20221231_143000.ogg',
 '20221231_213000.ogg',
 '20230101_000001.ogg',
 '20230101_060000.ogg',
 '20230101_143000.ogg',
 '20230101_213000.ogg',
 '20230102_000001.ogg',
 '20230102_060000.ogg',
 '20230102_143000.ogg',
 '20230102_213000.ogg',
 '20230103_000001.ogg',
 '20230103_060000.ogg',
 '20230103_213000.ogg',
 '20230104_000001.ogg',
 '20230104_060000.ogg',
 '20230104_143000.ogg',
 '20230104_213000.ogg',
 '20230105_000001.ogg',
 '20230105_060000.ogg',
 '20230105_143000.ogg',
 '20230105_213000.ogg',
 '20230106_000001.ogg',
 '20230106_060000.ogg',
 '20230106_143000.ogg',
 '20230106_213000.ogg',
 '20230107_000001.ogg',
 '20230107_060000.ogg',
 '20230107_143000.ogg',
 '20230107_213000.ogg',
 '20230108_000001.ogg']

In [29]:
generate_model_output('20230103_143000.ogg', sensor, model, config.device)

0 0.9983048439025879 random_other
2 0.9974689483642578 picus_viridis
5 0.9819239974021912 random_other
6 0.9553951025009155 picus_viridis
8 0.9761852622032166 picus_viridis
10 0.9707343578338623 picus_viridis
11 0.9812889099121094 picus_viridis
13 0.9986477494239807 picus_viridis
14 0.9981998205184937 random_other
15 0.993457019329071 picus_viridis
18 0.9725377559661865 picus_viridis
19 0.9983454942703247 picus_viridis
20 0.988653838634491 picus_viridis
21 0.9987242817878723 picus_viridis
22 0.9801102876663208 picus_viridis
23 0.9984299540519714 picus_viridis
24 0.9928951859474182 picus_viridis
25 0.9671839475631714 picus_viridis
28 0.9773991107940674 picus_viridis
29 0.9542649388313293 picus_viridis
31 0.977627694606781 picus_viridis
34 0.9746886491775513 random_other
35 0.9943013191223145 picus_viridis
37 0.9697950482368469 random_other
38 0.9950359463691711 picus_viridis
39 0.9966028928756714 picus_viridis
42 0.984915018081665 picus_viridis
47 0.9925175905227661 picus_viridis
48 0.9

KeyboardInterrupt: 